## 1 - Entendimento dos dados

In [10]:
from pathlib import Path
import pandas as pd

# Configuração específica para notebooks
PROJECT_ROOT = Path.cwd().parent.parent 
DATA_DIR = PROJECT_ROOT / "case_a3data" # Pasta raiz

# Verificação detalhada
try:
    csv_path = DATA_DIR / "books_data.csv"
    df_begin = pd.read_csv(csv_path)
    print(f"✅ Sucesso!")
    display(df_begin.head(4))
except FileNotFoundError:
    print(f"❌ Erro: Arquivo não encontrado em:\n{csv_path}")
    print("Verifique a estrutura de pastas:")
    print(f"Diretório atual: {Path.cwd()}")
    print(f"Raiz do projeto: {PROJECT_ROOT}")

✅ Sucesso!


,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
0,Its Only Art If Its Well Hung!,NaN,['Julie Strain'],http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,NaN,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,['Comics & Graphic Novels'],NaN
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],NaN
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],http://books.google.com/books/content?id=2tsDA...,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,NaN,2000,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,['Religion'],NaN
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],http://books.google.com/books/content?id=aRSIg...,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,iUniverse,2005-02,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,['Fiction'],NaN


In [11]:
print(df_begin.info(show_counts=True))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212404 entries, 0 to 212403
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Title          212403 non-null  object 
 1   description    143962 non-null  object 
 2   authors        180991 non-null  object 
 3   image          160329 non-null  object 
 4   previewLink    188568 non-null  object 
 5   publisher      136518 non-null  object 
 6   publishedDate  187099 non-null  object 
 7   infoLink       188568 non-null  object 
 8   categories     171205 non-null  object 
 9   ratingsCount   49752 non-null   float64
dtypes: float64(1), object(9)
memory usage: 16.2+ MB
None


In [12]:
# livro que não possui titulo (que por sinal, não possui descrição e nem categoria)
df_begin[df_begin['Title'].isna()]


,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
1066,NaN,NaN,['Maharshi Ramana'],http://books.google.com/books/content?id=uq0uj...,http://books.google.nl/books?id=uq0ujwEACAAJ&d...,NaN,2015-12-15,http://books.google.nl/books?id=uq0ujwEACAAJ&d...,NaN,1.0


## 2 - Limpeza dos dados

* Excluir Livros que não possuem Titulo
* Excluir colunas não uteis (image, previewLink, infoLink)
* padronizar a data de publicação (mantendo apenas o ano)
* Preencher valores nulos da coluna 'RatingScore' por 0.0
* Preencher valores nulos das demais colunas por 'not informed'

In [13]:
# Copiando o dataframe original
df_tratamento = df_begin.copy()

In [14]:
# Excluir Livros que não possuem Titulo
df_tratamento = df_tratamento[df_tratamento['Title'].notna()]


# Excluir colunas não uteis [image, previewLink, infoLink]
df_tratamento = df_tratamento.drop(columns=['image', 'previewLink', 'infoLink'])

In [15]:
df_tratamento.isnull().sum()

Title                 0
description       68441
authors           31413
publisher         75885
publishedDate     25305
categories        41198
ratingsCount     162652
dtype: int64

In [16]:
# Contar quantos livros não possuem nenhuma informação essencial (categories, description, authors, publisher)
total_livros = len(df_begin)

livros_sem_info = df_tratamento[
    df_tratamento['description'].isna() & 
    df_tratamento['authors'].isna() & 
    df_tratamento['publisher'].isna() &
    df_tratamento['categories'].isna()
].shape[0]

print(f'Quantidade de livros sem nenhuma informação essencial: {livros_sem_info}')
print(f'Porcentagem em relação ao total: {(livros_sem_info/total_livros*100):.2f}%')


df_tratamento[
    df_tratamento['categories'].isna() & 
    df_tratamento['description'].isna() & 
    df_tratamento['authors'].isna() & 
    df_tratamento['publisher'].isna()
].head(3)

Quantidade de livros sem nenhuma informação essencial: 25247
Porcentagem em relação ao total: 11.89%


,Title,description,authors,publisher,publishedDate,categories,ratingsCount
46,Cien Aos de BOXEO (One Hundred Years of Box),NaN,NaN,NaN,1864,NaN,NaN
151,Daughter of the Reef,NaN,NaN,NaN,NaN,NaN,NaN
152,The Scarlet Letter A Romance,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Nulos no ratingsCount será substituido por 0, e coluna será convertida para inteiro
df_tratamento['ratingsCount'] = df_tratamento['ratingsCount'].fillna(0)
df_tratamento['ratingsCount'] = df_tratamento['ratingsCount'].astype(int)

df_tratamento[df_tratamento['ratingsCount'] == 0].head(3)


,Title,description,authors,publisher,publishedDate,categories,ratingsCount
0,Its Only Art If Its Well Hung!,NaN,['Julie Strain'],NaN,1996,['Comics & Graphic Novels'],0
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],A&C Black,2005-01-01,['Biography & Autobiography'],0
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],NaN,2000,['Religion'],0


In [18]:
# Padronização do valor nulo em colunas de texto
null_default = 'Not informed'

# padronizar a data de publicação, mantendo apenas o ano
df_tratamento['publishedDate'] = (
    pd.to_datetime(df_tratamento['publishedDate'], errors='coerce')# converte a datas
      .dt.year                                          # extrai somente ano
      .astype('Int64')                                  # converção para inteiro, pois a conversão ficava como float
)
# Substituir valores nulos em description, authors, publisher, publishedDate e categories por "Not informed"
df_tratamento['publishedDate'] = df_tratamento['publishedDate'].astype(str).replace('<NA>', null_default)


# Substituir valores nulos em description, authors, publisher, e categories por "Not informed"
cols_para_substituir = ['description', 'authors', 'publisher', 'categories']
df_tratamento[cols_para_substituir] = df_tratamento[cols_para_substituir].fillna(null_default)


df_tratamento.head(5)


,Title,description,authors,publisher,publishedDate,categories,ratingsCount
0,Its Only Art If Its Well Hung!,Not informed,['Julie Strain'],Not informed,1996,['Comics & Graphic Novels'],0
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,['Philip Nel'],A&C Black,Not informed,['Biography & Autobiography'],0
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,['David R. Ray'],Not informed,2000,['Religion'],0
3,Whispers of the Wicked Saints,Julia Thomas finds her life spinning out of co...,['Veronica Haddon'],iUniverse,Not informed,['Fiction'],0
4,"Nation Dance: Religion, Identity and Cultural ...",Not informed,['Edward Long'],Not informed,Not informed,Not informed,0


In [19]:
# Quantidade valores nulos ('Not informed') em cada coluna, exceto 'ratingsCount'
cols_to_check = [col for col in df_tratamento.columns if col != 'ratingsCount']

for col in cols_to_check:
    count = (df_tratamento[col] == "Not informed").sum()
    print(f"A coluna '{col}' possui {count} valores 'Not informed'")




A coluna 'Title' possui 0 valores 'Not informed'
A coluna 'description' possui 68441 valores 'Not informed'
A coluna 'authors' possui 31413 valores 'Not informed'
A coluna 'publisher' possui 75885 valores 'Not informed'
A coluna 'publishedDate' possui 121452 valores 'Not informed'
A coluna 'categories' possui 41198 valores 'Not informed'


In [20]:
df_tratamento[21:24]


,Title,description,authors,publisher,publishedDate,categories,ratingsCount
21,Eyewitness Travel Guide to Europe,The DK Eyewitness Travel Guide: Eastern and Ce...,"['Dorling Kindersley Publishing Staff', 'Jonat...",Not informed,Not informed,['Europe'],0
22,Hunting The Hard Way,"Thrilling stories about hunting wildcat, buffa...",['Howard Hill'],Derrydale Press,Not informed,['Sports & Recreation'],0
23,History of Magic and the Occult,"See the history of witchcraft, magic and super...",['DK'],Dorling Kindersley Ltd,Not informed,"['Body, Mind & Spirit']",0


In [21]:
import ast
import numpy as np

def to_list_of_strings(value):
    """
    Converte 'value' em lista de strings, com os seguintes critérios:
    
    1. Se for "Not informed" ou NaN, retorna lista vazia.
    2. Se já for lista Python (list), retorna como está 
       (mas converte qualquer elemento que não seja string para string).
    3. Tenta interpretar como uma lista Python usando ast.literal_eval.
    4. Se ainda assim não for lista e contiver vírgula, faz split.
    5. Se for apenas uma string sem vírgula, retorna [essa_string].
    """
    
    # Se for "Not informed" ou NaN, retorna lista vazia
    if pd.isna(value) or value == "Not informed":
        return ["Not informed"]
    
    if isinstance(value, list):
        return [str(item).strip() for item in value]
    
    if isinstance(value, str) and value.startswith('[') and value.endswith(']'):
        try:
            parsed = ast.literal_eval(value)  # converte para list
            if isinstance(parsed, list):
                # garante que cada item seja string
                return [str(item).strip() for item in parsed]
            else:
                # se não for lista, apenas converte tudo para string
                return [str(parsed).strip()]
        except:
            # Se não conseguir interpretar, podemos tratar como string pura
            pass
    
    if isinstance(value, str):
        if ',' in value:
            return [v.strip() for v in value.split(',')]
        else:
            return [value.strip()]
    
    return [str(value).strip()]


df_tratamento['authors'] = df_tratamento['authors'].apply(to_list_of_strings)
df_tratamento['categories'] = df_tratamento['categories'].apply(to_list_of_strings)


In [22]:
has_multiple_authors = df_tratamento['authors'].apply(lambda x: len(x) > 1)
print("Alguma linha tem mais de um autor?", has_multiple_authors.any())

has_multiple_categories = df_tratamento['categories'].apply(lambda x: len(x) > 1)
print("Alguma linha tem mais de uma categoria?", has_multiple_categories.any())


Alguma linha tem mais de um autor? True
Alguma linha tem mais de uma categoria? False


In [23]:
# Define a raiz do projeto e a pasta de dados, conforme sua estrutura
PROJECT_ROOT = Path.cwd().parent.parent
SAVE_DIR = PROJECT_ROOT / "case_a3data" / "app" / "data"

# Cria a pasta, se ela não existir
SAVE_DIR.mkdir(parents=True, exist_ok=True)

# Define o caminho completo para o arquivo a ser salvo
csv_save_path = SAVE_DIR / "books_data_tratado.csv"

df_final = df_tratamento.copy()  
df_final.reset_index(drop=True, inplace=True)
df_final.to_csv(csv_save_path, index=False)
print(f"Arquivo salvo com sucesso em: {csv_save_path}")
df_final.head(3)

Arquivo salvo com sucesso em: c:\Users\Thiago_W\Desktop\A3_Case\case_a3data\app\data\books_data_tratado.csv


,Title,description,authors,publisher,publishedDate,categories,ratingsCount
0,Its Only Art If Its Well Hung!,Not informed,[Julie Strain],Not informed,1996,[Comics & Graphic Novels],0
1,Dr. Seuss: American Icon,Philip Nel takes a fascinating look into the k...,[Philip Nel],A&C Black,Not informed,[Biography & Autobiography],0
2,Wonderful Worship in Smaller Churches,This resource includes twelve principles in un...,[David R. Ray],Not informed,2000,[Religion],0


# FUTURAMENTE

* Futuramente, as categorias não informadas podem ser categorizadas utilizando llm, avaliando a descrição

- São contabilizados 41199 livros sem categoria (19.40% do total de livros)

In [222]:
# Contagem de livros sem categoria (ou seja, categoria == ["Not informed"])
sem_categoria = df_tratamento['categories'].apply(lambda x: x == ["Not informed"]).sum()
print(f'Quantidade de livros sem categoria: {sem_categoria}')

# Contagem de livros sem descrição (valor 'Not informed' como antes)
sem_descricao = (df_tratamento['description'] == 'Not informed').sum()
print(f'Quantidade de livros sem descrição: {sem_descricao}')

# Quantidade de livros sem categoria E sem descrição
sem_ambos = df_tratamento[
    (df_tratamento['categories'].apply(lambda x: x == ["Not informed"])) &
    (df_tratamento['description'] == 'Not informed')
].shape[0]
print(f'Quantidade de livros sem categoria E sem descrição: {sem_ambos}')

# Porcentagem em relação ao total
total_livros = len(df_tratamento)
print(f'\nTotal de livros no dataset: {total_livros}')
print(f'Porcentagem sem categoria: {(sem_categoria / total_livros * 100):.2f}%')
print(f'Porcentagem sem descrição: {(sem_descricao / total_livros * 100):.2f}%')
print(f'Porcentagem sem ambos: {(sem_ambos / total_livros * 100):.2f}%')


Quantidade de livros sem categoria: 41198
Quantidade de livros sem descrição: 68441
Quantidade de livros sem categoria E sem descrição: 35124

Total de livros no dataset: 212403
Porcentagem sem categoria: 19.40%
Porcentagem sem descrição: 32.22%
Porcentagem sem ambos: 16.54%
